# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [2]:
# Write your code below.

%load_ext dotenv
%dotenv 

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv
cannot find .env file


In [29]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [32]:
# Write your code below.
import os
from glob import glob

# Load the PRICE_DATA environment variable
price_data_dir = os.getenv('PRICE_DATA')
print(price_data_dir)

parquet_files = glob(os.path.join(price_data_dir, '*/*.parquet'))
parquet_files[:5]
dd_px = dd.read_parquet(parquet_files).set_index("ticker")

None


TypeError: expected str, bytes or os.PathLike object, not NoneType

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [33]:
import dask.dataframe as dd
from glob import glob

# Load the environment variable PRICE_DATA
price_data_dir = os.getenv('PRICE_DATA')

# Use glob to find the path of all parquet files in the directory PRICE_DATA
parquet_files = glob(os.path.join(price_data_dir, '*.parquet'))

# Read all the parquet files into a Dask DataFrame
df = dd.read_parquet(parquet_files)

# Function to add lagged values, returns, and range calculations
def add_features(data):
    data['Close_lag'] = data['Close'].shift(1)
    data['Adj_Close_lag'] = data['Adj Close'].shift(1)
    data['returns'] = (data['Adj Close'] / data['Adj_Close_lag']) - 1
    data['hi_lo_range'] = data['High'] - data['Low']
    return data

# Apply the function to each group of tickers
dd_feat = df.groupby('ticker').apply(add_features, meta={
    'Date': 'datetime64[ns]',
    'Open': 'float64',
    'High': 'float64',
    'Low': 'float64',
    'Close': 'float64',
    'Adj Close': 'float64',
    'Volume': 'int64',
    'sector': 'object',
    'subsector': 'object',
    'year': 'int64',
    'Close_lag': 'float64',
    'Adj_Close_lag': 'float64',
    'returns': 'float64',
    'hi_lo_range': 'float64'
})

dd_feat = dd_feat.persist()

# Display the first few rows to verify the results
dd_feat.head()


TypeError: expected str, bytes or os.PathLike object, not NoneType

+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [34]:
# Write your code below.
# Convert the Dask data frame to a pandas data frame
pd_feat = dd_feat.compute()

# Add rolling average return calculation with a window of 10 days
pd_feat['rolling_avg_return'] = pd_feat.groupby('ticker')['returns'].rolling(window=10).mean().reset_index(level=0, drop=True)

# Display the result
print(pd_feat.head())




NameError: name 'dd_feat' is not defined

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
No, it was not necessary. Dask DataFrames support many pandas operations, including calculating moving averages, directly on Dask objects. This means you can compute moving averages efficiently using Dask without needing to convert your data to a pandas DataFrame first.
+ Would it have been better to do it in Dask? Why?
Yes, it would have been better to calculate the moving average return in Dask, especially for handling large datasets such as stock market data. Dask leverages parallel computing, optimizing performance, and allows for handling datasets that are larger-than-memory, calculating moving averages efficiently.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.